In [ ]:
# %%
import os

print(os.getcwd())
os.chdir("..")
print(os.getcwd())

In [30]:
import numpy as np

fp = "modules/ManipNet/Code/Tensorflow/Data/ManipNetBIN/Encoder0_b0.bin"
array = np.fromfile(fp)
print(array.shape)
array = np.fromfile(fp, dtype=np.float32)
print(array.shape)
# print(array)

(256,)
(512,)


In [32]:
# %%
import os
import numpy as np
from pathlib import Path
import torch
from torch import nn




bindata = {}
binpath = "modules/ManipNet/Code/Tensorflow/Data/ManipNetBIN"
for fp in sorted(Path(binpath).iterdir()):
    # print(fp)
    bindata[fp.stem] = np.fromfile(fp, dtype=np.float32)
for k, v in bindata.items():
    print(k, v.shape)

# os.chdir("data/ManipNetBIN")
# print(os.getcwd())
# num = np.fromfile('Decoder_b0.bin')
# print(num)

# %%
class ResBlock(nn.Module):
    def __init__(self, in_vec, out_vec):
        super(ResBlock, self).__init__()
        self.FC = nn.Linear(in_vec, out_vec)
    def __call__(self, H_zero):
        H_zero = nn.ReLU(H_zero)
        H_one = self.FC(H_zero) + H_zero
        H_one  = nn.ReLU(H_one)
        H_two = self.res_fc(H_one) + H_one + H_zero
        return H_two

# %%
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.pose_fc = nn.Linear(192, 512)
        self.traj_fc = nn.Linear(336, 512)
        self.sensor_fc = nn.Linear(1822, 512)
        self.res0 = ResBlock(1536, 1536)
        self.res1 = ResBlock(1536, 1536)
        self.res2 = ResBlock(1536, 1536)
        self.decoder_fc = nn.Linear(1536, 214)

    def forward(self, pose, traj, sensor):
        x = self.flatten(x)
        pose = self.pose_fc(pose)
        traj = self.traj_fc(traj)
        sensor = self.sensor_fc(sensor)
        out = torch.cat(pose, traj, sensor, dim=1)
        out = self.res1(out)
        out = self.res2(out)
        out = self.decoder_fc(out)
        return out

model = NeuralNetwork().to(device)
# print(model)

# %%
# for name, param in model.named_parameters():
#     print(name, param.shape)

# # %%
# for k, v in bindata.items():
#     print(k, v.shape)

# %%
# with torch.no_grad():
#     for name, param in model.named_parameters():
#         # 重み名がNumPy配列のキーと一致するかを確認
#         print(name, param.flatten().shape)
        # for k, v in bindata.items():
        #     print(k, name)
        #     if name == "Decoder_b0.weight":
        #         print(name, param)
        #         param.copy_(torch.from_numpy(bindata["Decoder_b0"]))
        #         print("called")
        #         print(name, param)
        #     if name == "Decoder_w0.weight":
        #         print(name, param)
        #         param.copy_(torch.from_numpy(bindata["Decoder_w0"]))
        #         print("called")
        #         print(name, param)
        #         # 重みを適用（形状が一致している必要がある）
        #         # param.copy_(torch.from_numpy(bindata[k]))     

# %%
getbinname = {
    "decoder_fc.weight": "Decoder_w0",
    "decoder_fc.bias": "Decoder_b0",
    "pose_fc.weight": "Encoder0_w0",
    "pose_fc.bias": "Encoder0_b0",
    "traj_fc.weight": "Encoder1_w0",
    "traj_fc.bias": "Encoder1_b0",
    "sensor_fc.weight": "Encoder2_w0",
    "sensor_fc.bias": "Encoder2_b0",
    "res0.FC.weight": "DenseRes0_w0",
    "res0.FC.bias": "DenseRes0_b0",
    "res1.FC.weight": "DenseRes1_w0",
    "res1.FC.bias": "DenseRes1_b0",
    "res2.FC.weight": "DenseRes2_w0",
    "res2.FC.bias": "DenseRes2_b0",
}


with torch.no_grad():
    for name, param in model.named_parameters():
        binname = getbinname.get(name, None)
        if binname is not None:
            binweight = torch.from_numpy(bindata[binname])
            print(name, param.shape, param.flatten().shape, binname, binweight.shape)
        else:
            print(name, param.shape, param.flatten().shape)
        # for k, v in bindata.items():
        #     # print(name, k)
        #     if name == "Decoder_b0.weight":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Decoder_w0"]), [107, -1])
        #         # param.copy_(rephape_weight)
        #     if name == "Decoder_b0.bias":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Decoder_b0"]), [107])
        #         # param.copy_(rephape_weight)
        #     if name == "pose_fc.weight":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder0_w0"]), [256, 192])
        #         # param.copy_(rephape_weight)
        #     if name == "pose_fc.bias":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder0_b0"]), [256])
        #         # param.copy_(rephape_weight)
        #     if name == "traj_fc.weight":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder1_w0"]), [256, 336])
        #         # param.copy_(rephape_weight)
        #     if name == "traj_fc.bias":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder1_b0"]), [256])
        #         # param.copy_(rephape_weight)
        #     if name == "sensor_fc.weight":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder2_w0"]), [256, 1822])
        #         # param.copy_(rephape_weight)
        #     if name == "sensor_fc.bias":
        #         rephape_weight = torch.reshape(torch.from_numpy(bindata["Encoder2_b0"]), [256])
        #         # param.copy_(rephape_weight)
        #     print(name, param.shape, rephape_weight.shape)



Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)
Using cuda device
pose_fc.weight torch.Size([512, 192]) torch.Size([98304]) Encoder0_w0 torch.Size([98304])
pose_fc.bias torch.Size([512]) torch.Size([512]) Encoder0_b0 torch.Size([512])
traj_fc.weight torch.Size([512, 336]) torch.Size([172032]) Encoder1_w0 torch.Size([172032])
traj_fc.bias torch.Size([512]) torch.Size([512]) Encoder1_b0 torch.Size([512])
sensor_fc.weight torch.Size([512, 1822]) torch.Size([932864]) Encoder2_w0 torch.Size([932864])
sensor_fc.bias torch.Size([512]) torch.Size([512]) Encoder2_b0 torch.Size([512])
res0.FC.weight torch.Size([1536, 1536]) torch.Size([2359296]) DenseRes0_w